## Model Training, Evaluation and Optimization

In this section we load the input data, divide it into training set and testing set, set up cross-validation and tune hyperparameters, train the model, optimize it and evaluate its performance.

<center> Hyperparameter optimization details <center><br>

| Model | Hyperparameters | Fixed parameters |
|:-----:|:---------------:|:----------------:|
| PLS   | `n_components` = 1, 2, 4, 10 | – |
| RF    | `n_tree` = 500, 1000, 1500<br>`max_depth` = None, 5, 10 | – |
| SVR   | `C` = 1, 10, 100<br>`epsilon` = 0.1, 0.2, 0.5 | `kernel` = rbf |
| CNN   | `batch size` = 15, 20, 32<br>`layers` = [32, 64, 128]<br>`number of dense layers` = 1 or 2 | `optimizer` = adam<br>`objective` = mse, mae<br>`training epochs` = 500<br>`patience` = 20 |
| MLP   | `batch size` = 15, 20, 32<br>`layers` = [256, 128, 64]<br>`number of dense layers` = 1 | `optimizer` = adam<br>`objective` = mse, mae<br>`training epochs` = 500<br>`patience` = 5 |

In [ ]:
# Import dependecies

import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns

# scikit learn dependencies
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

# models
import sklearn
#import scikeras
from sklearn.cross_decomposition import PLSRegression # partial least squares regression
from sklearn.ensemble import RandomForestRegressor # random forest
from sklearn.svm import SVR # support vector regression

# to build neural networks (cnn, mlp)
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from scikeras.wrappers import KerasRegressor

# To estimate model performance
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error 
from scipy.stats import pearsonr

# For cross-validation 
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import LeaveOneOut

In [ ]:
# Load encoded dataset 
protein_sequences_file = 'aaindex_encoded.csv'  # CSV file path
df = pd.read_csv(protein_sequences_file)

In [ ]:
'''
# If columns (other that encoded data) need to be removed
df = df.drop(["sequence", "Fitness"], axis=1)
'''

In [ ]:
# prepare X variables i.e. encoded data

X = df.to_numpy() #convert to numpy array

In [ ]:
'''
# for AAindex, if encoded properties have not been combined

# First, convert the string lists to real lists
for col in ["CIDH920105", "KYTJ820101", "CHOP780201", "GRAR740102", 
            "HOPT810101", "ZIMJ680104","KARS160118","BUNA790103"]:
    df[col] = df[col].apply(ast.literal_eval)

# Now each cell in these columns is a real Python list, not a string!

# Next, flatten the lists row-wise
df['encoded_combined'] = df.apply(lambda row: np.concatenate([row[col] for col in [
    "CIDH920105", "KYTJ820101", "CHOP780201", "GRAR740102", 
    "HOPT810101", "ZIMJ680104","KARS160118","BUNA790103"
]]), axis=1)

# Finally, convert to a numeric matrix
X = np.stack(df['encoded_combined'].values)
'''

In [ ]:
'''# For Z-scales or other encodings, if the encoded data is not flattened 

# Convert the string representation of lists into actual lists
df["encoded_sequence"] = df["encoded_sequence"].apply(ast.literal_eval)

# Convert each sequence into a flattened numerical array
df["flattened_sequence"] = df["encoded_sequence"].apply(lambda x: np.array(x).flatten())

# Expand into a proper feature matrix
X = np.vstack(df["flattened_sequence"])
'''

In [ ]:
# load dataset with fitness values (in case it is in a different file or the column has been removed)
df2 = pd.read_csv("ExampleData.csv")

# prepare Y variable i.e fitness values

Y = df2["fitness"].to_numpy(dtype=np.float32) # fitness variable

In [ ]:
# To check shape of X and Y (num_samples, num_features) 

print("X Shape:", X.shape)  
print("Y Shape:", Y.shape)

In [ ]:
# Split data (adjust test_size, random_state constant)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
# Cross-validation using K-fold 

kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

def cross_validation(model, X, Y, cv):
    cv_score_r2 = cross_val_score(model, X, Y, scoring='r2', cv = cv)
    cv_score_mse = cross_val_score(model, X, Y, scoring='neg_mean_squared_error', cv = cv)
    cv_score_rmse = np.sqrt(-cv_score_mse)

    print(f"Cross-Validation R² scores: {cv_score_r2:.3f}")
    print(f"Mean R²: {cv_score_r2.mean():.3f}")
    print(f"Std of R²: {cv_score_r2.std():.5f}")
    print(" ")
    print(f"Cross-Validation RMSE scores: {cv_score_rmse:.5f}")
    print(f"Mean RMSE: {cv_score_rmse.mean():.5f}")
    print(f"Std of RMSE: {cv_score_rmse.std():.5f}")
    print(" ")

In [ ]:
# Model performance function

def model_performance(Y_train, Y_test, Y_train_pred, Y_test_pred):
    r2_train = r2_score(Y_train, Y_train_pred)
    r2_test = r2_score(Y_test, Y_test_pred)
    mse_train = mean_squared_error(Y_train, Y_train_pred)
    mse_test = mean_squared_error(Y_test, Y_test_pred)
    mae_train = mean_absolute_error(Y_train, Y_train_pred)
    mae_test = mean_absolute_error(Y_test, Y_test_pred)
    rmse_train = np.sqrt(mean_squared_error(Y_train, Y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(Y_test, Y_test_pred))

    print(f"Model Performance:")
    print(f"R² (Train): {r2_train:.3f}, R² (Test): {r2_test:.3f}")
    print(f"MAE (Train): {mae_train:.5f}, MAE (Test): {mae_test:.5f}")
    print(f"MSE (Train): {mse_train:.5f}, MSE (Test): {mse_test:.5f}")
    print(f"RMSE (Train): {rmse_train:.5f}, RMSE (Test): {rmse_test:.5f}")
    print(" ")

### Partial Least Squares (PLS) Regression 

In [ ]:
''' #In case standardization is needed

# Standardize Features for PLS 
scaler_X = StandardScaler()
scaler_Y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
Y_train = scaler_Y.fit_transform(Y_train)
Y_test = scaler_Y.transform(Y_test)
'''

# Training PLS Regression 

pls = PLSRegression(scale = True) 
pls_params = {'n_components': [1, 2, 4, 10]} # hyperparameters for grid search
pls_grid = GridSearchCV(pls, pls_params, scoring ='r2', cv = 5) 
pls_grid.fit(X_train, Y_train)

In [ ]:
# Get the best model
pls_best_model = pls_grid.best_estimator_

# Make predictions
Y_train_pred = pls_best_model.predict(X_train)
Y_test_pred = pls_best_model.predict(X_test)

'''
# Reverse Scaling (Optional: Convert Predictions Back to Original Scale)
Y_train_pred = scaler_Y.inverse_transform(Y_train_pred)
Y_test_pred = scaler_Y.inverse_transform(Y_test_pred)
'''

# Model Evaluation
model_performance(Y_train, Y_test, Y_train_pred, Y_test_pred)


In [ ]:
#To save GridSearchCV results 
results = pd.DataFrame(pls_grid.cv_results_)
results.to_csv('results_aaindex_pls.csv', index=False) # save as csv

# display(results[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]) # to display main results
# results # to display all results

### Random Forest (RF)

In [ ]:
# Training RF

rf = RandomForestRegressor()
rf_params = {'max_depth': [None, 5, 10], "n_estimators": [500, 1000, 1500]} # hyperparameters for grid search
rf_grid = GridSearchCV(rf, rf_params, cv = 5, scoring='r2', verbose = 1, error_score='raise')
rf_grid.fit(X_train, Y_train)

In [ ]:
# Get the best model
rf_best_model = rf_grid.best_estimator_

# Make predictions
Y_train_pred = rf_best_model.predict(X_train)
Y_test_pred = rf_best_model.predict(X_test)

# Model Evaluation
model_performance(Y_train, Y_test, Y_train_pred, Y_test_pred)


In [ ]:
#To save GridSearchCV results 
results = pd.DataFrame(rf_grid.cv_results_)
results.to_csv('results_aaindex_rf.csv', index=False)

# display(results[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]) # to display main results
# results # to display all results

### Support Vector Regression (SVR)

In [ ]:
''' #In case standardization is needed

# Standardize the features 
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# Optional: standardize target if it has large variance
scaler_Y = StandardScaler()
Y_scaled = scaler_Y.fit_transform(Y.reshape(-1, 1)).ravel()

# Split data (adjust test_size, keep random_state the same)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_scaled, test_size=0.3, random_state=42)
'''

In [ ]:
# Training SVR model
svr = SVR()
svr_params = {'C':[1, 10, 100], 'epsilon':[0.1, 0.2, 0.5], 'kernel':['rbf']} # hyperparameters for grid search
svr_grid = GridSearchCV(svr, svr_params, scoring='r2', cv=5, verbose=1)
svr_grid.fit(X_train, Y_train)

In [ ]:
# Get the best model
svr_best_model = svr_grid.best_estimator_

# Make predictions
Y_train_pred = svr_best_model.predict(X_train)
Y_test_pred = svr_best_model.predict(X_test)

# Model Evaluation
model_performance(Y_train, Y_test, Y_train_pred, Y_test_pred)


In [ ]:
#To save GridSearchCV results

results = pd.DataFrame(svr_grid.cv_results_)
results.to_csv('results_aaindex_svr_.csv', index=False)

# display(results[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]) # to display main results
# results # to display all results

### Convolution Neural Network (CNN)

In [ ]:
# Reshape input for CNN

'''
# if sequence length of encoded sequence is unknown
total_features = X.shape[1]        # total features
feat_dim = 5               # for example z-scales feature dimension is 5

seq_len = total_features // feat_dim
print("Sequence length:", seq_len)
'''

seq_len  = 145   
feat_dim = 5 # for example z-scales feature dimension is 5

X_train_cnn = X_train.reshape(-1, seq_len, feat_dim)  
X_test_cnn  = X_test .reshape(-1, seq_len, feat_dim)

In [ ]:
# Build CNN architecture

cnn = models.Sequential([
    layers.Input(shape=(seq_len, feat_dim)),      
    layers.Conv1D(32, kernel_size=5, activation="relu", padding="same"),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, kernel_size=3, activation="relu", padding="same"),
    layers.GlobalAveragePooling1D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(1)
])

cnn.compile(optimizer="adam", loss="mse", metrics=["mse", "mae"])

early = callbacks.EarlyStopping(patience=20, restore_best_weights=True)

history_cnn = cnn.fit(
    X_train_cnn, Y_train,
    epochs=300,
    batch_size=15,
    validation_split=0.1,
    callbacks=[early],
    verbose=1)


In [ ]:
# Make predictions

Y_train_pred = cnn.predict(X_train_cnn)
Y_test_pred = cnn.predict(X_test_cnn)

# Cross-validation and Model Evaluation

cross_validation(cnn, X, Y, kf)
model_performance(Y_train, Y_test, Y_train_pred, Y_test_pred)

### Multilayer Perceptron (MLP)

In [ ]:
# Build MLP architecture

# Optionally scale inputs
scaler = StandardScaler().fit(X_train)
X_train_mlp = scaler.transform(X_train)
X_test_mlp  = scaler.transform(X_test)


mlp = models.Sequential([
    layers.Input(shape=(X_train_mlp.shape[1],)),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(128, activation="sigmoid"),
    layers.Dropout(0.3),
    layers.Dense(64, activation="softmax"),
    layers.Dropout(0.3),
    layers.Dense(1)                     # regression output
])


mlp.compile(optimizer="adam", loss="mse", metrics=["mse", "mae"])

early = callbacks.EarlyStopping(patience=5, restore_best_weights=True)

history_mlp = mlp.fit(
    X_train_mlp, Y_train,
    epochs=300,
    batch_size=15,
    validation_split=0.1,
    callbacks=[early],
    verbose=1)


In [ ]:
# Make predictions

Y_train_pred = mlp.predict(X_train_mlp)
Y_test_pred = mlp.predict(X_test_mlp)

# Cross-validation and Model Evaluation

cross_validation(mlp, X, Y, kf)
model_performance(Y_train, Y_test, Y_train_pred, Y_test_pred)

In [ ]:
'''
# MLP using GridSearch # facing issues

start=time()

# define a function to create model, required for KerasClassifier
# the function takes drop_out rate as argument so we can optimize it  
def create_mlp_model(dropout_rate=0.3):
    # create model
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(X_train_mlp.shape[1],))) 
    # add a dropout layer if rate is not null
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))        
    model.add(Dense(128, activation='relu')) 
    # add a dropout layer if rate is not null    
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))           
    model.add(Dense(64, activation='sigmoid')) 
    # add a dropout layer if rate is not null    
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))           
    model.add(Dense(1, activation='softmax'))
    
    # Compile model
    model.compile( 
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=["mse", "mae", "accuracy"],
        )    
    return model
    
# define function to display the results of the grid search
def display_cv_results(search_results):
    print('Best score = {:.4f} using {}'.format(search_results.best_score_, search_results.best_params_))
    means = search_results.cv_results_['mean_test_score']
    stds = search_results.cv_results_['std_test_score']
    params = search_results.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print('mean test accuracy +/- std = {:.4f} +/- {:.4f} with: {}'.format(mean, stdev, param))    
    
# create model
model = KerasClassifier(build_fn=create_mlp_model, verbose=1)
# define parameters and values for grid search 
param_grid = {
    'batch_size': [16, 32, 64],
    'epochs': [n_epochs_cv],
    'dropout_rate': [0.0, 0.10, 0.20, 0.30],
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(X, to_categorical(Y))  # fit the full dataset as we are using cross validation 


display_cv_results(grid_result) # display full cv results
'''

In [ ]:
'''
# to check loss during neural network training

plt.subplot(1, 2, 2)
plt.plot(history_mlp.history['loss'], label='Training Loss')
plt.plot(history_mlp.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()
''''